# IEA CCS and PV forecasts: porcupine plots

Based on the work of Rory French. rory@koboldmetals.com.

Compiled by Lindsey Gulden, February 2025. lindsey@legupdata.com.

See explanation of methods [here](https://docs.google.com/document/d/16Hfk0xczmcDSuuA_6DvJqMnH7PelIWva9prwwlDKzv4/edit?usp=sharing).

See compiled data (with information and links to data sources) [here](https://docs.google.com/spreadsheets/d/1kxwt8ViqwPBqghgXq4PD39bd5_bLBUUiILu_P0IXmZ4/edit?usp=sharing).

### To run 
Download the Google Sheet linked above as an Excel workbook and point DATA_DIR to the local directory in which the Excel workbook is stored.


In [ ]:
DATA_DIR = "/Volumes/T5_external/data/porcupine"

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
import seaborn as sns
from pathlib import Path

import warnings

warnings.filterwarnings("ignore")
%load_ext jupyter_black



## Read data

In [ ]:
pv_actual_df = pd.read_excel(
    DATA_DIR / Path("CCS and PV projections.xlsx"), sheet_name="pv_actual"
)
pv_proj_df = pd.read_excel(
    DATA_DIR / Path("CCS and PV projections.xlsx"), sheet_name="pv_proj"
)
ccs_actual_df = pd.read_excel(
    DATA_DIR / Path("CCS and PV projections.xlsx"), sheet_name="ccs_actual"
)
ccs_proj_df = pd.read_excel(
    DATA_DIR / Path("CCS and PV projections.xlsx"), sheet_name="ccs_proj"
)

intensity_df = pd.read_csv(DATA_DIR / Path("carbon-intensity-electricity.csv"))

## Combine actual and projection data for plotting (both CCS and PV)

In [ ]:
# append a year's actual CCS capacity and solar PV electricity generation to each projection.

# PV
actual_data_for_all_df = (
    pv_proj_df[
        [
            "prediction_year",
            "which_prediction",
            "class",
            "scenario",
            "exclude_from_plot",
            "notes",
        ]
    ]
    .groupby(["prediction_year", "which_prediction"])
    .first()
    .reset_index()
    .merge(pv_actual_df, left_on="prediction_year", right_on="year")
)
actual_data_for_all_df["notes"] = "actual data for year prediction was issued"
actual_data_for_all_df["exclude_from_plot"] = 0
pv_df = pd.concat([pv_proj_df, actual_data_for_all_df])

# variable for hue identificaiton (plotting)
pv_df["proj_uid"] = [
    str(x) + "." + str(y) for x, y in zip(pv_df.prediction_year, pv_df.which_prediction)
]

# CCS
actual_data_for_all_df = (
    ccs_proj_df[
        [
            "prediction_year",
            "which_prediction",
            "class",
            "scenario",
            "exclude_from_plot",
            "notes",
        ]
    ]
    .groupby(["prediction_year", "which_prediction"])
    .first()
    .reset_index()
    .merge(ccs_actual_df, left_on="prediction_year", right_on="year")
)
actual_data_for_all_df["notes"] = "actual data for year prediction was issued"
actual_data_for_all_df["exclude_from_plot"] = 0
ccs_df = pd.concat([ccs_proj_df, actual_data_for_all_df])

ccs_df["proj_uid"] = [
    str(x) + "." + str(y)
    for x, y in zip(ccs_df.prediction_year, ccs_df.which_prediction)
]
ccs_df.to_clipboard()

## Porcupine Paper: plots for CCS and PV

In [ ]:
# add Gtco2 for plotting


# --- CCS CAPTURE CAPACITY ---
ccs_actual_df["gt_co2_per_yr"] = ccs_actual_df["mt_co2_per_yr"] / 1000
ccs_df["gt_co2_per_yr"] = ccs_df["mt_co2_per_yr"] / 1000
palette = "crest_r"  # "coolwarm"

marker = "."
font = "Arial Narrow"
weight = "semibold"
actualmarkersize = 12
actual_fontsize = 18
actuallinewidth = 2
ylabelsize = 12
startyear = 2005
endyear = 2035
projection_lw = 1.2
markersize = 11

title_fontsize = 18

fig, axes = plt.subplots(2, 1, figsize=(14, 10))
# --- CCS
ax = axes[0]
sns.lineplot(
    data=ccs_df.loc[
        # (ccs_df["prediction_year"] != 2011) &
        (ccs_df["exclude_from_plot"] == 0)
    ],
    x="year",
    y="gt_co2_per_yr",
    hue="proj_uid",
    errorbar=None,
    palette=palette,
    linewidth=projection_lw,
    linestyle="--",
    marker=marker,
    markersize=markersize,
    ax=ax,
)
ax.set_xlim([startyear, endyear])
ax.set_ylim([0, 1.750])
ax.grid()
ax.plot(
    ccs_actual_df["year"],
    ccs_actual_df["gt_co2_per_yr"],
    "k",
    marker=".",
    linewidth=actuallinewidth,
    markersize=actualmarkersize,
)

# ax.plot(solar_co2_df["year"], solar_co2_df["mt_co2_per_yr"], "r", marker=".")
h, l = ax.get_legend_handles_labels()
indices = [1, 4, 10, 13, 18]
for hh in h:
    hh.set_linewidth(projection_lw + 1)
ax.legend(
    title="Year of forecast",
    title_fontsize=14,
    fontsize=14,
    labels=[l[i][:4] for i in indices],
    handles=[h[i] for i in indices],
)
ax.tick_params(axis="x", which="major", labelsize=12)
ax.set_ylabel("Gt CO2/year", fontsize=ylabelsize)
ax.set_xlabel("")
# ax.legend(h[:5], l[:5], title="Year of forecast", title_fontsize=14, fontsize=14)

ax.text(
    x=2006,
    y=0.200,
    s="Actual operational CCS capture capacity",
    fontsize=actual_fontsize,
    font=font,
    weight=weight,
    path_effects=[pe.withStroke(linewidth=5, foreground="w", alpha=0.5)],
)
ax.set_title(
    "Global CO2 Capture Capacity: Forecasts vs. Reality", fontsize=title_fontsize
)

# --- SOLAR PV ---

ax = axes[1]
sns.lineplot(
    data=pv_df.loc[pv_df.exclude_from_plot == 0],
    x="year",
    y="world_solar_pv_twh",
    hue="proj_uid",
    errorbar=None,
    palette="crest_r",
    ax=ax,
    marker=".",
    linewidth=projection_lw,
    linestyle="--",
    markersize=markersize,
)
ax.set_xlim([startyear, endyear])
ax.set_ylim([0, 2250])
ax.grid()
ax.plot(
    pv_actual_df["year"],
    pv_actual_df["world_solar_pv_twh"],
    color="r",
    marker=".",
    linewidth=actuallinewidth,
    markersize=actualmarkersize,
)
ax.set_ylabel("TWh/year", fontsize=ylabelsize)
ax.set_xlabel("")
ax.text(
    x=2011.5,
    y=1250,
    s="Actual solar PV-generated electricity",
    fontsize=actual_fontsize,
    font=font,
    weight=weight,
    color="r",
    path_effects=[pe.withStroke(linewidth=5, foreground="w", alpha=0.5)],
)
h, l = ax.get_legend_handles_labels()
for hh in h:
    hh.set_linewidth(projection_lw + 1)
indices = [2, 6, 15, 27, 39]
ax.legend(
    title="Year of forecast",
    title_fontsize=14,
    fontsize=14,
    labels=[l[i][:4] for i in indices],
    handles=[h[i] for i in indices],
)
ax.tick_params(axis="x", which="major", labelsize=12)
ax.set_title(
    "Global Solar PV Electricity Generation: Forecasts vs. Reality",
    fontsize=title_fontsize,
)

fig.tight_layout()
fig.savefig("forecast_vs_reality_ccs_and_pv.png")
fig.savefig("forecast_vs_reality_ccs_and_pv.tif", format="tif")

## Hewlett presentation: Sequential figures for CCS projections

In [ ]:
plot_df = ccs_df.loc[
    # (ccs_df["prediction_year"] != 2011) &
    (ccs_df["exclude_from_plot"] == 0)
]

# --- CCS CAPTURE CAPACITY ---
ccs_actual_df["gt_co2_per_yr"] = ccs_actual_df["mt_co2_per_yr"] / 1000
ccs_df["gt_co2_per_yr"] = ccs_df["mt_co2_per_yr"] / 1000
palette = sns.color_palette(
    "crest_r", n_colors=plot_df.proj_uid.nunique()
)  # "coolwarm"

marker = "."
font = "Arial Narrow"
weight = "semibold"
actualmarkersize = 12
actual_fontsize = 18
actuallinewidth = 4
ylabelsize = 14
xtick_fontsize = 16
ytick_fontsize = 14
startyear = 2005
endyear = 2035
projection_lw = 3
markersize = 11

title_fontsize = 22


prediction_years = sorted(list(plot_df.prediction_year.unique()))
proj_uids = [float(x) for x in list(plot_df.prediction_year.unique())]
for which, py in enumerate(prediction_years):
    n_preds = len([x for x in proj_uids if x < py + 1])
    fig, ax = plt.subplots(1, 1, figsize=(16, 6))
    sns.lineplot(
        data=plot_df.loc[plot_df.prediction_year <= py],
        x="year",
        y="gt_co2_per_yr",
        hue="proj_uid",
        errorbar=None,
        palette=palette[: n_preds + 1],
        linewidth=projection_lw,
        # linestyle="--",
        marker=marker,
        markersize=markersize,
        ax=ax,
    )
    ax.set_xlim([startyear, endyear])
    ax.set_ylim([0, 1.750])

    ax.grid()
    if which == len(prediction_years) - 1:
        ax.plot(
            ccs_actual_df["year"],
            ccs_actual_df["gt_co2_per_yr"],
            marker=".",
            linewidth=actuallinewidth,
            color="r",
            markersize=actualmarkersize,
        )
        ax.text(
            x=2006,
            y=0.200,
            s="Actual operational CCS capture capacity",
            fontsize=actual_fontsize + 8,
            color="r",
            font=font,
            weight=weight,
            path_effects=[pe.withStroke(linewidth=5, foreground="w", alpha=0.5)],
        )
        # ax.plot(solar_co2_df["year"], solar_co2_df["mt_co2_per_yr"], "r", marker=".")
        h, l = ax.get_legend_handles_labels()
        h, l = ax.get_legend_handles_labels()
        indices = [1, 4, 10, 13, 18]
        for hh in h:
            hh.set_linewidth(projection_lw)
        ax.legend(
            title="Year of forecast",
            title_fontsize=16,
            fontsize=16,
            labels=[l[i][:4] for i in indices],
            handles=[h[i] for i in indices],
        )
    else:
        ax.text(
            x=2006,
            y=1.2500,
            s=f"{py}",
            fontsize=30,
        )
        ax.legend()
        ax.get_legend().remove()

    ax.tick_params(axis="x", which="major", labelsize=ytick_fontsize)
    ax.tick_params(axis="y", which="major", labelsize=xtick_fontsize)
    ax.set_ylabel("Gt CO2/year", fontsize=ylabelsize)
    ax.set_xlabel("")

    ax.set_title(
        "Global CO2 Capture Capacity: Forecasts vs. Reality", fontsize=title_fontsize
    )
    fig.savefig(f"{py}_ccs.png")

## Hewlett presentation comparison plots

In [ ]:
# add Gtco2 for plotting


# --- CCS CAPTURE CAPACITY ---
ccs_actual_df["gt_co2_per_yr"] = ccs_actual_df["mt_co2_per_yr"] / 1000
ccs_df["gt_co2_per_yr"] = ccs_df["mt_co2_per_yr"] / 1000
palette = "crest_r"  # "coolwarm"

marker = "."
font = "Arial Narrow"
weight = "semibold"
actualmarkersize = 14
actual_fontsize = 20
actuallinewidth = 4
ylabelsize = 16
startyear = 2005
endyear = 2035
projection_lw = 2.5
markersize = 11

title_fontsize = 24

fig, axes = plt.subplots(2, 1, figsize=(14, 8))
# --- CCS
ax = axes[0]
sns.lineplot(
    data=ccs_df.loc[
        # (ccs_df["prediction_year"] != 2011) &
        (ccs_df["exclude_from_plot"] == 0)
    ],
    x="year",
    y="gt_co2_per_yr",
    hue="proj_uid",
    errorbar=None,
    palette=palette,
    linewidth=projection_lw,
    marker=marker,
    markersize=markersize,
    ax=ax,
)
ax.set_xlim([startyear, endyear])
ax.set_ylim([0, 1.750])
ax.grid()
ax.plot(
    ccs_actual_df["year"],
    ccs_actual_df["gt_co2_per_yr"],
    "k",
    marker=".",
    linewidth=actuallinewidth,
    markersize=actualmarkersize,
)

# ax.plot(solar_co2_df["year"], solar_co2_df["mt_co2_per_yr"], "r", marker=".")
h, l = ax.get_legend_handles_labels()
indices = [1, 4, 10, 13, 18]
for hh in h:
    hh.set_linewidth(projection_lw + 1)
ax.legend(
    title="Year of forecast",
    title_fontsize=14,
    fontsize=14,
    labels=[l[i][:4] for i in indices],
    handles=[h[i] for i in indices],
)
ax.tick_params(axis="x", which="major", labelsize=16)
ax.tick_params(axis="y", which="major", labelsize=14)

ax.set_ylabel("Gt CO2/year", fontsize=ylabelsize)
ax.set_xlabel("")
# ax.legend(h[:5], l[:5], title="Year of forecast", title_fontsize=14, fontsize=14)

ax.plot(
    ccs_actual_df["year"],
    ccs_actual_df["gt_co2_per_yr"],
    marker=".",
    linewidth=actuallinewidth,
    color="r",
    markersize=actualmarkersize,
)
ax.text(
    x=2006,
    y=0.200,
    s="Actual operational CCS capture capacity",
    fontsize=actual_fontsize + 8,
    color="r",
    font=font,
    weight=weight,
    path_effects=[pe.withStroke(linewidth=5, foreground="w", alpha=0.5)],
)
ax.set_title(
    "Global CCS CO2-capture capacity: Forecasts vs. Reality",
    fontsize=title_fontsize,
)
# --- SOLAR PV ---

ax = axes[1]
sns.lineplot(
    data=pv_df.loc[pv_df.exclude_from_plot == 0],
    x="year",
    y="world_solar_pv_twh",
    hue="proj_uid",
    errorbar=None,
    palette="crest_r",
    ax=ax,
    marker=".",
    linewidth=projection_lw,
    markersize=markersize,
)
ax.set_xlim([startyear, endyear])
ax.set_ylim([0, 2250])
ax.grid()
ax.plot(
    pv_actual_df["year"],
    pv_actual_df["world_solar_pv_twh"],
    color="m",
    marker=".",
    linewidth=actuallinewidth,
    markersize=actualmarkersize,
)
ax.set_ylabel("TWh/year", fontsize=ylabelsize)
ax.set_xlabel("")
ax.text(
    x=2006.5,
    y=550,
    s="Actual solar PV electricity",
    fontsize=actual_fontsize + 8,
    font=font,
    weight=weight,
    color="m",
    path_effects=[pe.withStroke(linewidth=5, foreground="w", alpha=0.5)],
)
h, l = ax.get_legend_handles_labels()
for hh in h:
    hh.set_linewidth(projection_lw + 1)
indices = [2, 6, 15, 27, 39]
ax.legend(
    title="Year of forecast",
    title_fontsize=14,
    fontsize=14,
    labels=[l[i][:4] for i in indices],
    handles=[h[i] for i in indices],
)
ax.tick_params(axis="x", which="major", labelsize=16)
ax.tick_params(axis="y", which="major", labelsize=14)

ax.set_title(
    "Global Solar PV Electricity Generation: Forecasts vs. Reality",
    fontsize=title_fontsize,
)

fig.tight_layout()
fig.savefig("hewlett_forecast_vs_reality_ccs_and_pv.png")

## Carbon dioxide kept out of the atmosphere by solar pv electricity generation

In [ ]:
solar_co2_df = intensity_df.merge(
    pv_actual_df[["world_solar_pv_twh", "year"]], on="year"
)
solar_co2_df["mt_co2_per_yr"] = (
    (solar_co2_df["gco2_per_kwh"] / 1e6)
    * solar_co2_df["world_solar_pv_twh"]
    * 1e9
    / 1e6
)
solar_co2_df